In [1]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams.update({'font.size': 12})
pylab.rcParams.update({'font.family': 'sans serif'})
pylab.rcParams.update({'mathtext.default':'regular'})
pylab.rcParams.update({'mathtext.fontset':'stixsans'})
pylab.rcParams.update({'axes.linewidth': 1.5})
pylab.rcParams.update({'xtick.direction':'in'})
pylab.rcParams.update({'xtick.major.size': 5})
pylab.rcParams.update({'xtick.major.width': 1.25 })
pylab.rcParams.update({'xtick.minor.size': 2.5})
pylab.rcParams.update({'xtick.minor.width': 1.25 })
pylab.rcParams.update({'ytick.direction':'in'})
pylab.rcParams.update({'ytick.major.size': 5})
pylab.rcParams.update({'ytick.major.width': 1.25 })
pylab.rcParams.update({'ytick.minor.size': 2.5})
pylab.rcParams.update({'ytick.minor.width': 1.25 })

import warnings
warnings.filterwarnings("ignore")

T_colors = ["lightskyblue", "violet", "forestgreen", "darkviolet", "thistle", "cadetblue", 
            "palegoldenrod", "darksalmon", "indigo"]

Populating the interactive namespace from numpy and matplotlib


In [15]:
class SimUnitConverter:
    
    def __init__(self):
        # conversion factors to get CGS units into Cholla units
        self._LENGTH_UNIT = 3.08567758e21 # kpc in cm
        self._MASS_UNIT = 1.98855e33 # solar mass in grams
        self._TIME_UNIT = 3.15569e10 # kyr in s
        self._DENSITY_UNIT = (self._MASS_UNIT/(self._LENGTH_UNIT**3))
        self._VELOCITY_UNIT = (self._LENGTH_UNIT/self._TIME_UNIT)
        self._PRESSURE_UNIT = (self._DENSITY_UNIT*self._VELOCITY_UNIT**2)

    def density(self, d_cgs):
        # convert g/cm^3 to M_sun/kpc^3
        return d_cgs / self._DENSITY_UNIT
    
    def mass(self, m_cgs):
        # convert g to M_sun
        return m_cgs / self._MASS_UNIT
    
    def time(self, t_cgs):
        # convert s to kyr
        return t_cgs / self._TIME_UNIT
    
    def length(self, l_cgs):
        # convert cm to kpc
        return l_cgs / self._LENGTH_UNIT
    
    def velocity(self, v_cgs):
        # convert cm/s to kpc/kyr
        return v_cgs / self._VELOCITY_UNIT

    def pressure(self, p_cgs):
        # convert g*cm^-1*s^-2 to M_sun*kpc^-1*kyr^-2
        return p_cgs / self._PRESSURE_UNIT

7.387289587479254e+31

In [ ]:
class SputteringIntegrator:
    MP = 1.6726E-24;  # proton mass in g
    YR_IN_S = 3.154e7 # one year in s
    # use this ^ conversion just because it's easier to think about time in terms of years
    
    def __init__(self, T, n, dt, tmax):
        self.T = T
        self.n = n
        self.dt = dt # gets passed in in s
        self._d0_gas, self._d0_dust = self.calc_init_density()
        self.d_gas, self.d_dust = self.calc_init_density()
        self.tau_sp = self.calc_tau_sp() # s
        self.type = "Sputtering"

    def calc_tau_sp(self):
        a1 = 1; # dust grain size in units of 1 micrometer
        d0 = 1; # gas density in units of 10^-27 g/cm^3
        T_0 = 2e6; # K
        omega = 2.5;
        A = 0.17e9 * self.YR_IN_S; # 0.17 Gyr in s
        
        tau_sp = A * (a1/d0) * ((T_0/self.T) ** omega + 1); # s
        
        return tau_sp
    
    def calc_init_density(self):
        d0_gas = self.MP * self.n # g/cm^3
        # assume 1% dust to gas fraction
        d0_dust = d0_gas / 100
        
        return d0_gas, d0_dust
    
    def calc_d_dust(self):
        dd_dt = self.calc_dd_dt()
        d_dust_i = self.d_dust + self.dt * dd_dt
        
        return d_dust_i
        
    def calc_dd_dt(self):
        dd_dt = -self.d_dust/(self.tau_sp/3) # g/cm^3/s
        
        return dd_dt
    
    def update_densities(self):
        dd_dt = self.calc_dd_dt()
        self.d_dust += self.dt * dd_dt # g/cm^3
    
    def get_info(self):
        print("Sputtering")
        print("------------------------------------------")
        print("Gas temperature: {:.1E} K".format(self.T))
        print("Gas initial density: {:.5E} M_sun/kpc^3".format(self._d0_gas))
        print("Destruction timescale: {:.5E} yr".format(self.tau_sp/self.YR_IN_S))
        print("Time-step: {} yr".format(self.dt/self.YR_IN_S))
        print("------------------------------------------")

In [ ]:
# analytic solution to sputtering DE
def thermal_sputtering(t, T, h, n, tmax):
    a1 = 1; # dust grain size in units of 1 micrometer
    d0 = 1; # gas density in units of 10^-27 g/cm^3
    T_0 = 2e6; # K
    omega = 2.5;
    A = 0.17e9; # 0.17 Gyr

    tau_sp = A * (a1/d0) * ((T_0/T) ** omega + 1); # yr
    
    integrator_i = SputteringIntegrator(h, n, [T], tmax)
    d0_dust = integrator_i._d0_dust
    
    return d0_dust * np.exp(-3*t/tau_sp)

In [ ]:
def evolve_solutions(h, n, T_arr, tmax):
    YEAR_IN_SECONDS = 3.154e7
    h *= YEAR_IN_SECONDS
    tmax *= YEAR_IN_SECONDS
    
    t_arr = np.arange(0, tmax, h)
    
    d_dust = np.zeros(shape=(len(T_arr), len(t_arr)))

    integrators = []
    
    taus = []
    
    def time_refine(integrator):
        """Refine time-step if dd is changing too rapidly."""
        d_dust_i = integrator.calc_d_dust()
        dd_dt = integrator.calc_dd_dt()
        dt = integrator.dt
        dd = dd_dt * dt
        while dd/d_dust_i > 0.01:
            dt_sub = 0.01 * d_dust_i / dd_dt

            integrator.d_dust += dt_sub * dd_dt
            integrator.dt -= dt_sub
            dt = integrator.dt
            dd_dt = integrator.calc_dd_dt()
            dd = dt * dd_dt

    for j, T in enumerate(T_arr):
        integrator = SputteringIntegrator(T, n, h, tmax)
        integrator.get_info()
        # set initial densities
        d_dust[j][0] = integrator._d0_dust
        taus.append(integrator.tau_sp) # s
        integrators.append(integrator)

        for i, t_i in enumerate(t_arr):
            if (i+1) < len(t_arr):
                for integrator in integrators:
                    # calculate dust density for this time-step
                    d_dust_i = integrator.calc_d_dust()
                    # calculate dd
                    dd_dt = integrator.calc_dd_dt()
                    dd = integrator.dt * dd_dt

                    # if rate of dust growth is changing too rapidly
                    if dd/d_dust_i > 0.01:
                        # shorten time-step
                        time_refine(integrator)
                        integrator.dt = h
                        d_dust[j][i+1] = integrator.d_dust
                        continue

                    # this is where the density attribute gets updated
                    integrator.update_densities()
                    d_dust[j][i+1] = integrator.d_dust

    return d_dust, taus, integrators

In [ ]:
T_arr = [1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8]

h = 1000
n = 1e-2
tmax = 1e9

dd, tau_sp, integrators = evolve_solutions(h, n, T_arr, tmax)

In [ ]:
# time step
t_arr = np.arange(0, tmax, h)

fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7,10))
for j, T in enumerate(T0_cells):
    ax[0].semilogx(t_arr, dd[j], linestyle="--", color="grey")
    ax[0].semilogx(t_arr, thermal_sputtering(t_arr, T, h, n, tmax), color=T_colors[j], linewidth=4, alpha=0.5, zorder=0)
    ax[1].semilogx(t_arr, thermal_sputtering(t_arr, T, h, n, tmax) - dd[j], color=T_colors[j], label=f"$T_0={T:.0E}~K$")
    ax[0].vlines(tau_sp[j]/integrators[j].YR_IN_S, ymin=np.amin(dd[j]), ymax=np.amax(dd[j]), color="k", linestyle="--", zorder=0)
    ax[1].set_xlabel("Time$~[yr]$")
    ax[0].set_ylabel(r"Dust Density$~[g/cm^3]$")
    ax[1].set_ylabel("analytic - numerical")
    ax[1].legend()
ax[0].set_title("thermal sputtering of dust")
ax[0].set_xlim(0, 1e9)
# plt.savefig("/Users/helenarichie/GitHub/helenarichie.github.io/assets/img/posts/sputtering_numerical.png", facecolor="white")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,6))
for j, T in enumerate(T0_cells):
    ax.semilogx(t_arr, dd[j], linestyle="--", linewidth=2, color="darkgrey")
    ax.semilogx(t_arr, thermal_sputtering(t_arr, T, h, n, tmax), color=T_colors[j], linewidth=4, alpha=0.5, zorder=0, label=f"$T_0={T:.0E}~K$")
    ax.set_xlabel("Time$~[yr]$")
    ax.set_ylabel(r"Dust Density$~[M_\odot/kpc^3]$")
    ax.legend()
ax.set_title("thermal sputtering of dust")
#plt.savefig("/Users/helenarichie/GitHub/helenarichie.github.io/assets/img/posts/sputtering_numerical.png", facecolor="white")